<a href="https://colab.research.google.com/github/JackSD/Data-Science-Work/blob/master/GoogleVisionAPI_video_annotating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Google Vision API to object-annotate a video

In [0]:
import cv2
from google.cloud import vision
from google.cloud import storage
from google.cloud.vision import types
import glob
import numpy as np
import os
import io

#authenticate with Google Cloud
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/jackd/googlecloudacc/My Test Project-d2230389d5ee.json"
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

#from video, get frames
def generate_images(video_name):
  vidcap = cv2.VideoCapture('resources/' + video_name)
  success,image = vidcap.read()
  count = 0
  while success:
    cv2.imwrite("images/frame" + '{0:03}'.format(count) + '.jpg', image)     # save frame as JPEG file
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1

#annotate the frames using the API (object detection)
def annotate_images(folder):
  client = vision.ImageAnnotatorClient()
  directory = "C:/Users/jackd/PycharmProjects/video_processing/images/"
  output_directory = "C:/Users/jackd/PycharmProjects/video_processing/output_images/"
  files = os.listdir(directory)

  for file_name in files:
    print(file_name)
    with io.open(directory + file_name, 'rb') as image_file:
      content = image_file.read()

    image = types.Image(content=content)

    objects = client.object_localization(
      image=image).localized_object_annotations
    print('Number of objects found: {}'.format(len(objects)))

    #now annotate the image
    image = cv2.imread(directory + file_name)
    color_blue = (255, 0, 0)

    for someobject in objects:
      x1 = round(someobject.bounding_poly.normalized_vertices[0].x * image.shape[1])
      y1= round(someobject.bounding_poly.normalized_vertices[0].y * image.shape[0])
      x2 = round(someobject.bounding_poly.normalized_vertices[2].x * image.shape[1])
      y2 = round(someobject.bounding_poly.normalized_vertices[2].y * image.shape[0])
      label = someobject.name
      cv2.rectangle(image, (x1, y1), (x2, y2), color_blue, 2)
      # Write text on the image
      cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_blue, 2)

    #cv2.imshow("image", image)
    #cv2.waitKey(0)

    cv2.imwrite(output_directory + file_name,image)

def rebuild_video():
  output_directory = "C:/Users/jackd/PycharmProjects/video_processing/output_images/"
  img_array = []
  for filename in glob.glob(output_directory + '*.jpg'):
    print(filename)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

  out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

if __name__ == '__main__':
    generate_images('protest_vid.mp4')
    annotate_images('images')
    rebuild_video()